<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/doublebinaryoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities (assuming flat_rate_py, flat_vol_py, exercise_type_to_string_py,
# payoff_type_to_string_py, time_to_days_py are defined as in previous examples)

def flat_rate_py(evaluation_date_or_quote, forward_rate_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        forward_rate_obj = forward_rate_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        forward_rate_obj = evaluation_date_or_quote
        day_counter = forward_rate_or_dc
    if isinstance(forward_rate_obj, ql.Quote):
        quote_handle = ql.QuoteHandle(forward_rate_obj)
    elif isinstance(forward_rate_obj, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate_obj))
    else:
        quote_handle = forward_rate_obj
    return ql.FlatForward(evaluation_date, quote_handle, day_counter)

def flat_vol_py(evaluation_date_or_quote, vol_level_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        vol_level_obj = vol_level_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        vol_level_obj = evaluation_date_or_quote
        day_counter = vol_level_or_dc
    if isinstance(vol_level_obj, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_level_obj)
    elif isinstance(vol_level_obj, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_level_obj))
    else:
        vol_quote_handle = vol_level_obj
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, day_counter)

def time_to_days_py(t, basis=360):
    return int(t * basis + 0.5)

def exercise_type_to_string_py(exercise):
    if isinstance(exercise, ql.EuropeanExercise): return "European"
    if isinstance(exercise, ql.AmericanExercise): return "American"
    return "UnknownExercise"

def payoff_type_to_string_py(payoff):
    if isinstance(payoff, ql.PlainVanillaPayoff): return "PlainVanilla"
    if isinstance(payoff, ql.CashOrNothingPayoff): return "CashOrNothing"
    # Add other payoff types if needed
    return "UnknownPayoff"

class DoubleBinaryOptionData:
    def __init__(self, barrierType, barrier_lo, barrier_hi, cash, s, q, r, t, v, result, tol):
        self.barrierType = barrierType
        self.barrier_lo = barrier_lo
        self.barrier_hi = barrier_hi
        self.cash = cash
        self.s = s; self.q = q; self.r = r; self.t = t; self.v = v
        self.result = result; self.tol = tol

class DoubleBinaryOptionTests(unittest.TestCase):
    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary but fixed
        ql.Settings.instance().evaluationDate = self.today

        self.dc = ql.Actual360()
        self.spot_q = ql.SimpleQuote(0.0)
        self.q_rate_q = ql.SimpleQuote(0.0)
        self.r_rate_q = ql.SimpleQuote(0.0)
        self.vol_q = ql.SimpleQuote(0.0)

        self.spot_h = ql.QuoteHandle(self.spot_q)
        self.qTS_h = ql.YieldTermStructureHandle()
        self.rTS_h = ql.YieldTermStructureHandle()
        self.volTS_h = ql.BlackVolTermStructureHandle()

        self._link_market_data(self.today)
        self.process = ql.BlackScholesMertonProcess(self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def _link_market_data(self, eval_date):
        self.qTS_h.linkTo(flat_rate_py(eval_date, self.q_rate_q, self.dc))
        self.rTS_h.linkTo(flat_rate_py(eval_date, self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(eval_date, self.vol_q, self.dc))

    def _update_market_data(self, current_eval_date, s, q, r, v):
        ql.Settings.instance().evaluationDate = current_eval_date
        self.spot_q.setValue(s)
        self.q_rate_q.setValue(q)
        self.r_rate_q.setValue(r)
        self.vol_q.setValue(v)
        self._link_market_data(current_eval_date)

    def _report_failure_binary(self, test_name_prefix, payoff, exercise, barrier_type,
                               barrier_lo, barrier_hi, s, q, r, today, v,
                               expected, calculated, error, tolerance):
        # Determine option type string from payoff if possible, or use generic
        option_type_str = "Binary" # Default
        if hasattr(payoff, 'optionType'): # CashOrNothingPayoff has it
            option_type_str = str(payoff.optionType())

        msg = (f"{option_type_str} option with {barrier_type} barrier type (Test: {test_name_prefix}):\n"
               f"    barrier_lo:       {barrier_lo}\n    barrier_hi:       {barrier_hi}\n"
               f"    Payoff:           {payoff_type_to_string_py(payoff)}\n"
               f"    spot value:       {s}\n"
               # Strike for CashOrNothing is part of payoff, C++ test used 0 for strike in payoff
               f"    strike (payoff):  {payoff.strike() if hasattr(payoff, 'strike') else 'N/A'}\n"
               f"    dividend yield:   {q:.4f}\n    risk-free rate:   {r:.4f}\n"
               f"    reference date:   {today}\n    maturity:         {exercise.lastDate()}\n"
               f"    volatility:       {v:.4f}\n\n"
               f"    expected   value: {expected}\n    calculated value: {calculated}\n"
               f"    error:            {error}\n    tolerance:        {tolerance}")
        self.fail(msg)

    def test_haug_values(self):
        print("Testing cash-or-nothing double barrier options against Haug's values...")
        # Using a subset of Haug's values from C++ for brevity
        values = [
            DoubleBinaryOptionData(ql.DoubleBarrier.KnockOut, 80.00, 120.00, 10.00, 100.00, 0.02, 0.05, 0.25, 0.10, 9.8716, 1e-4),
            DoubleBinaryOptionData(ql.DoubleBarrier.KnockOut, 95.00, 105.00, 10.00, 100.00, 0.02, 0.05, 0.25, 0.50, 0.0000, 1e-4),
            DoubleBinaryOptionData(ql.DoubleBarrier.KIKO,    80.00, 120.00, 10.00, 100.00, 0.02, 0.05, 0.25, 0.10, 0.0000, 1e-4),
            DoubleBinaryOptionData(ql.DoubleBarrier.KnockIn, 90.00, 110.00, 10.00, 100.00, 0.02, 0.05, 0.25, 0.30, 9.0798, 1e-4), # KI from VBA
            # Degenerate cases
            DoubleBinaryOptionData(ql.DoubleBarrier.KnockOut, 95.00, 105.00, 10.00,  80.00, 0.02, 0.05, 0.25, 0.10, 0.0000, 1e-4),
            DoubleBinaryOptionData(ql.DoubleBarrier.KnockIn,  95.00, 105.00, 10.00, 110.00, 0.02, 0.05, 0.25, 0.10, 10.0000, 1e-4), # Note: Haug's book has 10 for KI if S is outside
        ]
        current_eval_date = ql.Date(10, ql.July, 2020) # Example fixed date

        for val_data in values:
            self._update_market_data(current_eval_date, val_data.s, val_data.q, val_data.r, val_data.v)

            # The C++ test uses CashOrNothingPayoff(Option::Call, 0, value.cash)
            # This means it's a call option with strike 0, paying fixed cash.
            # For a double barrier binary, this effectively means if spot is between L and H at expiry, pay cash.
            # The "Option::Call" with strike 0 means it's always "in the money" if active.
            payoff = ql.CashOrNothingPayoff(ql.Option.Call, 0.0, val_data.cash) # Effectively pays cash if not knocked out / if knocked in

            ex_date = current_eval_date + time_to_days_py(val_data.t)
            exercise = None
            if val_data.barrierType == ql.DoubleBarrier.KIKO or val_data.barrierType == ql.DoubleBarrier.KOKI:
                exercise = ql.AmericanExercise(current_eval_date, ex_date)
            else:
                exercise = ql.EuropeanExercise(ex_date)

            option = ql.DoubleBarrierOption(val_data.barrierType, val_data.barrier_lo, val_data.barrier_hi,
                                            0.0, # rebate
                                            payoff, exercise)

            # AnalyticDoubleBarrierBinaryEngine
            engine_analytic_binary = ql.AnalyticDoubleBarrierBinaryEngine(self.process)
            option.setPricingEngine(engine_analytic_binary)
            calculated_analytic = option.NPV()
            error_analytic = abs(calculated_analytic - val_data.result)
            if error_analytic > val_data.tol:
                self._report_failure_binary("AnalyticBinary", payoff, exercise, val_data.barrierType,
                                     val_data.barrier_lo, val_data.barrier_hi, val_data.s, val_data.q, val_data.r,
                                     current_eval_date, val_data.v, val_data.result,
                                     calculated_analytic, error_analytic, val_data.tol)

            # BinomialDoubleBarrierEngine
            # try:
            #     # This engine might not be directly suitable for binary payoffs if not specialized,
            #     # or might require specific tree setup.
            #     engine_binomial = ql.BinomialDoubleBarrierEngine(self.process, "CoxRossRubinstein", 500) # Steps
            #     option.setPricingEngine(engine_binomial)
            #     calculated_binom = option.NPV()
            #     error_binom = abs(calculated_binom - val_data.result)
            #     tol_binom = 0.22 # C++ tolerance
            #     if error_binom > tol_binom:
            #         self._report_failure_binary("Binomial", payoff, exercise, val_data.barrierType,
            #                             val_data.barrier_lo, val_data.barrier_hi, val_data.s, val_data.q, val_data.r,
            #                             current_eval_date, val_data.v, val_data.result,
            #                             calculated_binom, error_binom, tol_binom)
            # except Exception as e:
            #     print(f"Skipping BinomialDoubleBarrierEngine test for binary due to: {e}")


    def test_pde_double_barrier_with_analytical(self):
        print("Testing cash-or-nothing double barrier (PDE Heston vs Analytic BS)...")

        current_eval_date = ql.Date(30, ql.January, 2023)
        maturity_date = current_eval_date + ql.Period(1, ql.Years)
        ql.Settings.instance().evaluationDate = current_eval_date

        spot_val = 100.0
        r_val = 0.075
        q_val = 0.03
        vol_val = 0.4

        # Use distinct market data objects for this test to avoid interference
        spot_h_pde = ql.QuoteHandle(ql.SimpleQuote(spot_val))
        dc_pde = ql.Actual360() # As per C++ test
        rTS_h_pde = ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, r_val, dc_pde))
        qTS_h_pde = ql.YieldTermStructureHandle(flat_rate_py(current_eval_date, q_val, dc_pde))

        # Heston parameters for BS degeneracy
        kappa = 1.0
        theta = vol_val**2
        rho = 0.0
        sigma_vol_of_vol = 1e-4 # Near zero
        v0 = theta # Start at long-term variance

        heston_process = ql.HestonProcess(rTS_h_pde, qTS_h_pde, spot_h_pde, v0, kappa, theta, sigma_vol_of_vol, rho)
        heston_model = ql.HestonModel(heston_process)

        # BS Process for analytic engine
        volTS_h_bs = ql.BlackVolTermStructureHandle(flat_vol_py(current_eval_date, vol_val, dc_pde))
        bs_process = ql.BlackScholesMertonProcess(spot_h_pde, qTS_h_pde, rTS_h_pde, volTS_h_bs)

        analytic_engine = ql.AnalyticDoubleBarrierBinaryEngine(bs_process)
        fd_engine = ql.FdHestonDoubleBarrierEngine(heston_model, 201, 101, 3, 0, ql.FdmSchemeDesc.Hundsdorfer())

        european_exercise = ql.EuropeanExercise(maturity_date)
        # Payoff: CashOrNothing(Call, 0.0, 1.0) -> pays 1 if not KO and S>0 (always true if S>0)
        # This effectively tests a "Double-No-Touch" that pays 1 if barriers are not hit.
        binary_payoff = ql.CashOrNothingPayoff(ql.Option.Call, 0.0, 1.0)

        tolerance = 5e-3

        for i in range(5, 10): # Reduced loop for speed, C++ goes to 18 (5 to 17 by 2)
            dist = 10.0 + 5.0 * i
            barrier_lo = max(spot_val - dist, 1e-2)
            barrier_hi = spot_val + dist

            double_barrier_option = ql.DoubleBarrierOption(
                ql.DoubleBarrier.KnockOut, barrier_lo, barrier_hi, 0.0, # rebate
                binary_payoff, european_exercise
            )

            double_barrier_option.setPricingEngine(analytic_engine)
            bs_npv = double_barrier_option.NPV()

            double_barrier_option.setPricingEngine(fd_engine)
            fd_npv = double_barrier_option.NPV()

            diff = fd_npv - bs_npv
            self.assertLessEqual(abs(diff), tolerance,
                                 msg=f"PDE Double-No-Touch vs BS: Lo={barrier_lo}, Hi={barrier_hi}\n"
                                     f"BS NPV: {bs_npv:.4f}, FD Heston NPV: {fd_npv:.4f}, Diff: {diff:.4e}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)